In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon July 16 10:08:53 2018

@author: sxx
"""
import time
import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import traceback 

from predict import load_data as load_data
from predict import transform_data as transform_data
from predict import format_data as format_data
from predict import divide_data as divide_data
from predict import LstmModel as LstmModel

Using TensorFlow backend.
/opt/anaconda3/envs/health/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
current_time=time.strftime('%Y-%m-%d-%H-%M',time.localtime())

def experiments(params):
    print ("epoch:",params['epoch']," batch_size: ",params['batch'])
    csvfile = './data/health_179.csv'
    x,y = load_data(csvfile)
    x_,y_ = transform_data(x,y,False)
    datax,datay = format_data(x_,y_)
    trainX,trainY,validX,validY,testX,testY = divide_data(datax,datay)
    
    try:
        ##调试模型epoch,batch_size,learnrate
        #model = LstmModel(epoches=params['epoch'], batch_size=params['batch'],lr=params['lr'])

        ##测试模型units,dropout_rate
        model = LstmModel(units=params['units'],dr=params['dr'],lr=params['lr'])
        health_model = model.create_model()
        save_path = './models/health_{0}.h5'.format(current_time)
        model.train_model_1(trainX,trainY,validX,validY,save_path)
        testPredict = health_model.predict(testX)
        print (testY,"-->",testPredict)
    except Exception as e:
        print ("something happen:",repr(e))
        print ('-' * 20)
        with open("./data/err.log","a") as logf:
            traceback.print_exc(file=logf)
        return {'loss': 999999, 'status': STATUS_OK}
    
    mse = np.mean(np.square(testPredict - testY))
    print ("mse:",mse)
    print ('-' * 20)

    return {'loss': mse, 'status': STATUS_OK}


In [3]:

if __name__ == '__main__':
    space = {
        'units': {'lstm_units':{
        'units1': hp.choice('units1', [i for i in range(120, 160,2)]),
        'units2': hp.choice('units2', [i for i in range(80, 120,2)]),
        'units3': hp.choice('units3', [i for i in range(40, 80,2)]),
        'units4': hp.choice('units4', [i for i in range(20, 40,2)])},
        'dense_units':{
        'units5': hp.choice('units5', [i for i in range(6, 20,2)]),
        'units6': 5}
        },
        'dr': hp.uniform('dr',0.1,0.6),
        'batch': hp.randint('batch',256),
        'epoch': hp.randint('epoch',500),
        'lr': hp.uniform('lr',0.0001,0.01),
        'activation': hp.choice('activation',['relu',
                                                'sigmoid',
                                                'tanh',
                                                'linear'])
        }
    
    trials = Trials()
    best = fmin(experiments, space, algo=tpe.suggest, max_evals=50, trials=trials)
    print ('best: ',best)
    print ("hyperoptparam end")

epoch: 236  batch_size:  236
LSTM model unit1_size:150 unit2_size:96 unit3_size:56 unit4_size:30 unit5_size:12 unit6_size:5

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 15, 150)           121800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 150)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 150)           600       
_________________________________________________________________
lstm_2 (LSTM)                (None, 15, 56)            46368     
_________________________________________________________________
dropout_2 (Dropout)          (None, 15, 56)            0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 56)            224       
_________________

17199/17199 [==============================] - 4s 231us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00035: val_loss did not improve from 0.00107
Epoch 36/446
17199/17199 [==============================] - 4s 230us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00036: val_loss did not improve from 0.00107
Epoch 37/446
17199/17199 [==============================] - 4s 231us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00037: val_loss did not improve from 0.00107
Epoch 38/446
17199/17199 [==============================] - 4s 231us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00038: val_loss did not improve from 0.00107
Epoch 39/446
17199/17199 [==============================] - 4s 236us/step - loss: 0.0014 - val_loss: 0.0012

Epoch 00039: val_loss did not improve from 0.00107
Epoch 40/446
17199/17199 [==============================] - 4s 235us/step - loss: 0.0014 - val_loss: 0.0012

Epoch 00040: val_loss did not improve from 0.00107
Epoch 41/446
17199/17199 [===========================

Train on 17199 samples, validate on 4914 samples
Epoch 1/446
17199/17199 [==============================] - 6s 326us/step - loss: 0.2164 - val_loss: 0.0021

Epoch 00001: val_loss did not improve from 0.00107
Epoch 2/446
17199/17199 [==============================] - 4s 235us/step - loss: 0.0020 - val_loss: 0.0020

Epoch 00002: val_loss did not improve from 0.00107
Epoch 3/446
17199/17199 [==============================] - 4s 236us/step - loss: 0.0019 - val_loss: 0.0011

Epoch 00003: val_loss did not improve from 0.00107
Epoch 4/446
17199/17199 [==============================] - 4s 235us/step - loss: 0.0018 - val_loss: 0.0011

Epoch 00004: val_loss did not improve from 0.00107
Epoch 5/446
17199/17199 [==============================] - 4s 236us/step - loss: 0.0017 - val_loss: 0.0011

Epoch 00005: val_loss did not improve from 0.00107
Epoch 6/446
17199/17199 [==============================] - 4s 235us/step - loss: 0.0017 - val_loss: 0.0012

Epoch 00006: val_loss did not improve from 0.001

17199/17199 [==============================] - 4s 235us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00052: val_loss did not improve from 0.00107
Epoch 53/446
17199/17199 [==============================] - 4s 234us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00053: val_loss did not improve from 0.00107
Epoch 54/446
17199/17199 [==============================] - 4s 234us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00054: val_loss did not improve from 0.00107
Epoch 55/446
17199/17199 [==============================] - 4s 235us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00055: val_loss did not improve from 0.00107
Epoch 56/446
17199/17199 [==============================] - 4s 235us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00056: val_loss did not improve from 0.00107
Epoch 57/446
17199/17199 [==============================] - 4s 234us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00057: val_loss did not improve from 0.00107
Epoch 58/446
17199/17199 [===========================

17199/17199 [==============================] - 4s 235us/step - loss: 0.0016 - val_loss: 0.0011

Epoch 00007: val_loss did not improve from 0.00107
Epoch 8/446
17199/17199 [==============================] - 4s 235us/step - loss: 0.0015 - val_loss: 0.0011

Epoch 00008: val_loss did not improve from 0.00107
Epoch 9/446
17199/17199 [==============================] - 4s 236us/step - loss: 0.0015 - val_loss: 0.0011

Epoch 00009: val_loss did not improve from 0.00107
Epoch 10/446
17199/17199 [==============================] - 4s 235us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00010: val_loss did not improve from 0.00107
Epoch 11/446
17199/17199 [==============================] - 4s 235us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00011: val_loss did not improve from 0.00107
Epoch 12/446
17199/17199 [==============================] - 4s 236us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00012: val_loss did not improve from 0.00107
Epoch 13/446
17199/17199 [=============================

...

17199/17199 [==============================] - 4s 261us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00052: val_loss did not improve from 0.00105
Epoch 53/446
17199/17199 [==============================] - 4s 260us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00053: val_loss did not improve from 0.00105
Epoch 54/446
17199/17199 [==============================] - 4s 260us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00054: val_loss did not improve from 0.00105
Epoch 55/446
17199/17199 [==============================] - 4s 261us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00055: val_loss did not improve from 0.00105
Epoch 56/446
17199/17199 [==============================] - 4s 260us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00056: val_loss did not improve from 0.00105
Epoch 57/446
17199/17199 [==============================] - 4s 260us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00057: val_loss did not improve from 0.00105
Epoch 58/446
17199/17199 [===========================

17199/17199 [==============================] - 4s 260us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00016: val_loss did not improve from 0.00105
Epoch 17/446
17199/17199 [==============================] - 4s 260us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00017: val_loss did not improve from 0.00105
Epoch 18/446
17199/17199 [==============================] - 4s 260us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00018: val_loss did not improve from 0.00105
Epoch 19/446
17199/17199 [==============================] - 4s 260us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00019: val_loss did not improve from 0.00105
Epoch 20/446
17199/17199 [==============================] - 4s 261us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00020: val_loss did not improve from 0.00105
Epoch 21/446
17199/17199 [==============================] - 4s 260us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00021: val_loss did not improve from 0.00105
Epoch 22/446
17199/17199 [===========================

17199/17199 [==============================] - 4s 260us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00118: val_loss did not improve from 0.00105
Epoch 119/446
17199/17199 [==============================] - 4s 260us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00119: val_loss did not improve from 0.00105
Epoch 120/446
17199/17199 [==============================] - 4s 260us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00120: val_loss did not improve from 0.00105
Epoch 121/446
17199/17199 [==============================] - 4s 259us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00121: val_loss did not improve from 0.00105
Epoch 122/446
17199/17199 [==============================] - 4s 259us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00122: val_loss did not improve from 0.00105
Epoch 123/446
17199/17199 [==============================] - 4s 260us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00123: val_loss did not improve from 0.00105
Epoch 124/446
17199/17199 [=====================

17199/17199 [==============================] - 4s 260us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00169: val_loss did not improve from 0.00105
Epoch 170/446
17199/17199 [==============================] - 4s 260us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00170: val_loss did not improve from 0.00105
Epoch 171/446
17199/17199 [==============================] - 4s 259us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00171: val_loss did not improve from 0.00105
Epoch 172/446
17199/17199 [==============================] - 4s 259us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00172: val_loss did not improve from 0.00105
Epoch 173/446
17199/17199 [==============================] - 4s 261us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00173: val_loss did not improve from 0.00105
Epoch 174/446
17199/17199 [==============================] - 4s 259us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00174: val_loss did not improve from 0.00105
Epoch 00174: early stopping
[[0.95   0.9275 0.93

17199/17199 [==============================] - 5s 263us/step - loss: 0.0013 - val_loss: 0.0011

Epoch 00030: val_loss did not improve from 0.00105
Epoch 31/446
17199/17199 [==============================] - 4s 262us/step - loss: 0.0014 - val_loss: 0.0011

Epoch 00031: val_loss did not improve from 0.00105
Epoch 32/446
17199/17199 [==============================] - 5s 263us/step - loss: 0.0013 - val_loss: 0.0026

Epoch 00032: val_loss did not improve from 0.00105
Epoch 33/446
17199/17199 [==============================] - 5s 262us/step - loss: 0.0013 - val_loss: 0.0027

Epoch 00033: val_loss did not improve from 0.00105
Epoch 34/446
17199/17199 [==============================] - 4s 261us/step - loss: 0.0016 - val_loss: 0.0011

Epoch 00034: val_loss did not improve from 0.00105
Epoch 35/446
17199/17199 [==============================] - 5s 262us/step - loss: 0.0014 - val_loss: 0.0012

Epoch 00035: val_loss did not improve from 0.00105
Epoch 36/446
17199/17199 [===========================